In [1]:
import gurobipy as grb
import numpy as np

opt_model = grb.Model(name="MIP Model")
opt_model.setParam("NonConvex", 2)
opt_model.setParam("TimeLimit", 1000)

Academic license - for non-commercial use only - expires 2021-03-26
Using license file /home/nick/gurobi.lic
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [2]:
# get data
from dataset import MovieLensDataset
d = MovieLensDataset('/home/nick/datasets/gurobi_data/', mode='sparse')
# d = MovieLensDataset('data/', mode='sparse')

Loading MovieLens dataset from /home/nick/datasets/gurobi_data/ with mode sparse..
Column names are 1, 1, 4.0, 964982703
Processed 1140 lines. 10 users x 868 movies.
Dataset contains 1139 ratings (13.122119815668205% matrix density)


In [3]:
rows, cols = d.X.nonzero()
ratings = d.X.data

In [ ]:
# def miqp(features, response, non_zero, warm_up=None, verbose=False):
regressor = grb.Model()
samples, dim = features.shape

# Append a column of ones to the feature matrix to account for the y-intercept
X = np.concatenate([features, np.ones((samples, 1))], axis=1)  

# Decision variables
u = regressor.addVars(d.n_users, lb=-grb.GRB.INFINITY, name="users")
v = regressor.addVars(d.n_movies, lb=-grb.GRB.INFINITY, name="movies")

# intercept = beta[dim] # Last decision variable captures the y-intercept
# intercept.varname = 'intercept'
# iszero[i] = 1 if beta[i] = 0  
# iszero = regressor.addVars(dim, vtype=GRB.BINARY, name="iszero") 

# Objective Function (OF): minimize 1/2 * RSS using the fact that
# if x* is a minimizer of f(x), it is also a minimizer of k*f(x) iff k > 0
Quad = np.dot(X.T, X)
lin = np.dot(response.T, X)
obj = sum(0.5 * Quad[i,j] * beta[i] * beta[j]
          for i, j in product(range(dim + 1), repeat=2))
obj -= sum(lin[i] * beta[i] for i in range(dim + 1))
obj += 0.5 * np.dot(response, response)
regressor.setObjective(obj, GRB.MINIMIZE)

# Constraint sets
for i in range(dim):
    # If iszero[i]=1, then beta[i] = 0
    regressor.addSOS(GRB.SOS_TYPE1, [beta[i], iszero[i]])
regressor.addConstr(iszero.sum() == dim - non_zero) # Budget constraint

# We may use the Lasso or prev solution with fewer features as warm start
if warm_up is not None and len(warm_up) == dim:
    for i in range(dim):
        iszero[i].start = (abs(warm_up[i]) < 1e-6)

if not verbose:
    regressor.params.OutputFlag = 0
regressor.params.timelimit = 60
regressor.params.mipgap = 0.001
regressor.optimize()

coeff = np.array([beta[i].X for i in range(dim)])
return intercept.X, coeff     


In [5]:
u = opt_model.addVars(d.n_users, lb=-20.0, ub=20.0, vtype=grb.GRB.CONTINUOUS, name="users")
v = opt_model.addVars(d.n_movies,lb=-20.0, ub=20.0, vtype=grb.GRB.CONTINUOUS, name="movies")
# theta = opt_model.addVars(d.n_users, d.n_movies, vtype=grb.GRB.INTEGER)
# x_vars  ={(i,j):opt_model.addVar(vtype=grb.GRB.INTEGER,
#                         name="x_{0}_{1}".format(i,j)) for i in rows for j in cols}

In [6]:
# objective = grb.quicksum((u[i]*v[j] - d.X[i,j])**2
#                          for i, j in zip(rows, cols))
# objective = sum((u[rows[i]]*v[cols[i]] - d.X[rows[i], cols[i]]) for i in range(len(ratings)))
objective = 0
for i, j in zip(rows, cols):
    z = opt_model.addVar(name=f'z_{i}_{j}')
    opt_model.addConstr(u[i]*v[j] == z)
    objective += (z - d.X[i, j]) * (z - d.X[i, j])


In [7]:
opt_model.setObjective(objective, grb.GRB.MINIMIZE)
opt_model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 0 rows, 2895 columns and 0 nonzeros
Model fingerprint: 0xa2641f81
Model has 1139 quadratic objective terms
Model has 1139 quadratic constraints
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 878 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 0 rows and 878 columns
Presolve time: 0.00s
Presolved: 4556 rows, 2017 columns, 13668 nonzeros
Presolved model has 1139 quadratic objective terms
Presolved model has 1139 bilinear constraint(s)
Variable types: 2017 continuous, 0 integer (0 binary)

Root relaxation: objective 0.000000e+00, 1703 iterations, 0.04 seconds

    N

 504266 294773  798.94981   33 1139 1221.51339  372.44509  69.5%  15.6  350s
 510749 299918  504.68617   51 1139 1221.51339  373.25123  69.4%  15.6  355s
 518899 305902     cutoff   39      1221.51339  373.94587  69.4%  15.7  360s
 525770 311113  671.22342   35 1138 1221.51339  374.61911  69.3%  15.6  365s
 534325 316086 1217.78253   71 1139 1221.51339  376.63946  69.2%  15.6  370s
 542164 321539     cutoff   46      1221.51339  378.06481  69.0%  15.6  375s
 551165 327907     cutoff   55      1221.51339  378.50592  69.0%  15.5  381s
 558318 332518  789.15823   74 1139 1221.51339  381.28287  68.8%  15.5  385s
 566613 337745     cutoff   68      1221.51339  381.81907  68.7%  15.5  390s
 574562 344149 1130.41155  214 1139 1221.51339  382.19106  68.7%  15.5  395s
 580968 348180  563.06482   61 1139 1221.51339  384.03358  68.6%  15.6  400s
 588898 353370 1095.78772   76 1084 1221.51339  384.59110  68.5%  15.6  405s
 595674 356939 1136.53647   48 1139 1221.51339  385.73493  68.4%  15.6  410s

 1326448 855490 1079.70861  135 1139 1221.51339  450.21072  63.1%  15.2  880s
 1334939 861631 1134.94618   96 1139 1221.51339  450.37389  63.1%  15.1  885s
 1344054 868446  581.81812   60 1139 1221.51339  450.83353  63.1%  15.1  890s
 1350450 872986  663.81911   56 1139 1221.51339  451.11716  63.1%  15.1  895s
 1358490 879124 1171.79474  107 1139 1221.51339  451.30257  63.1%  15.1  900s
 1367760 885860  998.66304   77 1139 1221.51339  452.03859  63.0%  15.1  905s
 1374889 890389 1081.62425   97 1139 1221.51339  452.41081  63.0%  15.1  910s
 1382987 895809 1156.56706   77 1139 1221.51339  452.94917  62.9%  15.1  915s
 1384581 896612  790.03297   79 1139 1221.51339  452.94917  62.9%  15.1  920s
 1393430 903390 1109.93081   91 1139 1221.51339  453.59443  62.9%  15.1  926s
 1400572 907563  888.73649   58 1139 1221.51339  454.00570  62.8%  15.1  930s
 1408201 913722 1197.63543  186 1139 1221.51339  454.30129  62.8%  15.1  935s
 1416654 919568 1163.68989  108 1139 1221.51339  454.52665  62.8

In [8]:
u_arr = np.array([u[i].X for i in range(d.n_users)])
v_arr = np.array([v[i].X for i in range(d.n_movies)])
v_arr

array([ -8.18388824,  -7.29237027,  -7.20661246,  -7.43690214,
        -5.34891862,  -8.91486485,  -7.52111552,  -8.22617504,
        -8.91486485,  -7.33360278,  -8.22617504,  -6.19015515,
        -7.40311249,  -5.79689858,  -9.75355697,  -5.51511228,
        -6.30251755,  -8.6028293 ,  -7.54911139,  -8.18427523,
        -7.33360278,  -7.04765071,  -5.34891862,  -4.98564511,
        -8.41933978,  -8.46345535,  -6.14632967,  -8.09303598,
        -6.40282828,  -6.40282828,  -7.73427862,  -8.89941303,
        -6.1362439 ,  -8.3963487 ,  -8.12264624,  -7.68482584,
        -6.4738574 ,  -8.91486485,  -5.34891862,  -7.13189199,
        -7.25106745,  -7.81104675,  -7.13189199,  -9.53682485,
        -8.83137047,  -8.91486485,  -7.13189199,  -8.91486485,
        -5.34891862,  -8.91486485,  -8.91486485,  -8.65556029,
        -8.91486485,  -5.34891862,  -8.91486485,  -8.91486485,
        -6.40282828,  -9.1166986 ,  -5.86691833,  -7.60529581,
        -9.53682485,  -8.91486485,  -7.13490177,  -8.91

In [10]:
v_arr.shape
u_arr.shape

(10,)

In [11]:
u_arr = u_arr.reshape(-1, 1)
v_arr = v_arr.reshape(-1, 1)

# let's now evaluate the MEAN squared error (above we're just dealing with sum of squared errors)
from main import evaluate
mse = evaluate(u_arr, v_arr, d.X, 'sparse')
mse

(10, 1) (868, 1) (10, 868) sparse
Test set MSE: 1.0724438106457805


1.0724438106457805

In [17]:
# let's see whether fun eval matches
from als import ALSSparse
als = ALSSparse(u=u_arr, v=v_arr, dataset=d.X)
als.function_eval()

1221.513500325544